In [1]:
#set path to data set
train_path = 'data/classification/{}/train/' #pass the plant name as variable
valid_path = 'data/classification/{}/valid/'
model_weights = 'weights/disease classification - {}.h5'

batch_size = 32 #Number of samples  passed through to the network at one time.
valid_size = 16
color_mode = 'rgb'

width = 224
height = 224

target_size = (width, height) #input image size
input_shape = (width, height, 3) #he shape of the input data provided to the Keras model while training

zoom_range = 0.3
shear_range = 0.3
shift_range = 0.3
rotation_range = 30

val_split = 0.2

dense_1 = 512
dense_2 = 256
dense_3 = 64

epochs = 50
rate = 0.2

verbose = 1

In [2]:
import os, json
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import cv2 as cv
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.activations import relu
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
############################################################################################

#used mobilent_v2 model for image pre-processing
def preprocessing_function(img):
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
    return img

def image_data_generator(plant):
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                                                    rotation_range = rotation_range,
                                                    shear_range = shear_range,
                                                    zoom_range = zoom_range,
                                                    width_shift_range=shift_range,
                                                    height_shift_range=shift_range,
                                                    horizontal_flip = True,
                                                    preprocessing_function=preprocessing_function
                                                                )
    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                                                    preprocessing_function=preprocessing_function
                                                                )

    train_generator = train_datagen.flow_from_directory(
                                    train_path.format(plant),
                                    target_size = target_size,
                                    color_mode = color_mode,
                                    batch_size = batch_size,
                                    class_mode = 'categorical',
                                    shuffle = True
                                    )

    validation_generator = train_datagen.flow_from_directory(
                                    valid_path.format(plant),
                                    target_size = target_size,
                                    color_mode = color_mode,
                                    batch_size = valid_size,
                                    class_mode = 'categorical',
                                    shuffle = True
                                    )

    return train_generator, validation_generator

In [3]:
#setup the parameter and performance metrics.

class DiseaseClassification(object):
    def __init__(self, plant, train_generator, validation_generator):
        self.train_generator = train_generator
        self.validation_generator = validation_generator
        self.train_step = self.train_generator.samples // batch_size
        self.validation_step = self.validation_generator.samples // valid_size

        self.accuracy = tf.keras.metrics.BinaryAccuracy()
        self.recall = tf.keras.metrics.Recall()
        self.precision = tf.keras.metrics.Precision()

        self.id2disease = {v:k for k, v in self.train_generator.class_indices.items()}
        self.plant = plant

        self.num_calsses = len(self.id2disease)
#Fine tune the mobilenet v2 model by adding new layers to model
    def classifier(self, x):
        if not self.trainable:
            x = Dense(dense_1, activation='relu')(x)
            x = Dense(dense_1)(x)
            x = BatchNormalization()(x)
            x = relu(x)                                          
            x = Dropout(rate)(x)

            x = Dense(dense_2, activation='relu')(x)
            x = Dense(dense_2)(x)
            x = BatchNormalization()(x)
            x = relu(x)
            x = Dropout(rate)(x)

        x = Dense(dense_3, activation='relu')(x)
        x = Dense(dense_3)(x)
        x = BatchNormalization()(x)
        x = relu(x)
        x = Dropout(rate)(x)
        return x

    def model_conversion(self, trainable):
        functional_model = tf.keras.applications.MobileNetV2(weights="imagenet")
        functional_model.trainable = trainable

        self.trainable = trainable

        inputs = functional_model.input

        x = functional_model.layers[-2].output
        x = self.classifier(x)
        outputs = Dense(self.num_calsses, activation='softmax')(x)

        model = Model(
                inputs=inputs,
                outputs=outputs
                    )
                    
        self.model = model
        self.model.summary()

    def train(self):
        callback = tf.keras.callbacks.EarlyStopping(                  #Method use for prevent model overfitting
                                                monitor='loss', 
                                                patience=4
                                                    )

        self.model.compile(                                       #How to optimize model
                          optimizer='Adam',
                          loss='categorical_crossentropy',        #for model error calculation
                          metrics=[                                
                                self.accuracy,
                                self.recall,
                                self.precision
                                  ]
                          )
        self.model.fit(
                    self.train_generator,
                    steps_per_epoch= self.train_step,
                    validation_data= self.validation_generator,
                    validation_steps = self.validation_step,
                    epochs=epochs,
                    verbose=verbose
                        )

    def save_model(self):
        self.model.save(model_weights.format(self.plant))             

    def load_model(self):
        self.model = load_model(model_weights.format(self.plant))
        self.model.compile(
                          optimizer='Adam',
                          loss='categorical_crossentropy',
                          metrics=[
                                self.accuracy,
                                self.recall,
                                self.precision
                                  ]
                         )

    def predict(self, x):                          #Get the predication
        x = preprocessing_function(x)
        x = np.expand_dims(x, axis=0)
        P = self.model.predict(x)
        Pest_id = np.argmax(P)
        disease = self.id2disease[Pest_id]         #get the class name using index value
        return disease
        
    def process(self):
        if not os.path.exists(model_weights.format(self.plant)):
            self.model_conversion(False)
            self.train()
            self.save_model()
        else:
            self.load_model()

# APPLE

In [4]:
train_generator, validation_generator = image_data_generator('apple')
train_generator.class_indices

Found 7771 images belonging to 4 classes.
Found 1943 images belonging to 4 classes.


{'Apple___Apple_scab': 0,
 'Apple___Black_rot': 1,
 'Apple___Cedar_apple_rust': 2,
 'Apple___healthy': 3}

In [5]:
model = DiseaseClassification('apple', train_generator, validation_generator)
model.process()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

c:\Users\Legion\.conda\envs\tf26\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [7]:
train_generator, validation_generator = image_data_generator('bell_pepper')
train_generator.class_indices

Found 3901 images belonging to 2 classes.
Found 975 images belonging to 2 classes.


{'Pepper,_bell___Bacterial_spot': 0, 'Pepper,_bell___healthy': 1}

In [8]:
model = DiseaseClassification('bell_pepper', train_generator, validation_generator)
model.process()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
____________________________________________________________________________________________

In [9]:
train_generator, validation_generator = image_data_generator('corn')
train_generator.class_indices

Found 7316 images belonging to 4 classes.
Found 1829 images belonging to 4 classes.


{'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': 0,
 'Corn_(maize)___Common_rust_': 1,
 'Corn_(maize)___Northern_Leaf_Blight': 2,
 'Corn_(maize)___healthy': 3}

In [10]:
model = DiseaseClassification('corn', train_generator, validation_generator)
model.process()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
____________________________________________________________________________________________

In [11]:
train_generator, validation_generator = image_data_generator('potato')
train_generator.class_indices

Found 5702 images belonging to 3 classes.
Found 1426 images belonging to 3 classes.


{'Potato___Early_blight': 0, 'Potato___Late_blight': 1, 'Potato___healthy': 2}

In [12]:
model = DiseaseClassification('potato', train_generator, validation_generator)
model.process()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
____________________________________________________________________________________________